In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [ ]:
##reading the data set
stroke_df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
stroke_df.head()

In [ ]:
##getting the rough idea about the null values
plt.subplots(figsize = (25,10))
sns.heatmap(stroke_df.isna(),yticklabels=False)
plt.show()

In [ ]:
stroke_df.info()

In [ ]:
stroke_df.isna().sum()

In [ ]:
##getting the shape of data frame
print('The shape of the data is {}'.format(stroke_df.shape))

In [ ]:
stroke_df.dtypes

In [ ]:
#Analysin 'gender' column
stroke_df.gender.value_counts()

In [ ]:
##since only one count is for other category, lets drop it 
other = stroke_df.loc[stroke_df.gender=='Other'].index
stroke_df.drop(other,axis = 0,inplace =True)

In [ ]:
stroke_df.age.describe()

Minimum age is 0.08, having a look at it

In [ ]:
stroke_df[stroke_df.age ==.08]

In [ ]:
#analysis of 'work_type' variable
stroke_df.work_type.value_counts()


* 687 are children and never worked is 22,

* since children is not a suitable entry for work_type so making a column which will be holding the values for the age caregory

* editing the value of children as Never_worked in work_type variable

In [ ]:
stroke_df[stroke_df.work_type == 'children'].age.describe()

In [ ]:
stroke_df[stroke_df.work_type == 'Never_worked'].age.describe()

Creating a variable that will store the value of age category for example child, youth, adult, senior

In [ ]:
 #creating conditions
condition = [
    (stroke_df['age'] <=14),
    (stroke_df['age'] > 14) & (stroke_df['age'] <=25),
    (stroke_df['age'] > 25) & (stroke_df['age'] <=60),
    (stroke_df['age'] > 60)
    ]
#creating list of values
values = ['child','youth','adult','senior']

#creating a new column
stroke_df['Age_category'] = np.select(condition,values)

#getting the head of df
stroke_df.head()

In [ ]:
#changing the value of data in work_type col
stroke_df.work_type = np.where(stroke_df['work_type'] == 'children','Never_worked',stroke_df.work_type)
stroke_df.work_type.value_counts()

In [ ]:
##treating the null values of bmi column
stroke_df[stroke_df.bmi.isnull()]

In [ ]:
##creating the function which will give the bmi according to category in variable
def bmi_finder(variable):
    for category in stroke_df[variable].unique():
        print('bmi for category = ' + str(category))
        print(stroke_df[stroke_df[variable] == str(category)].bmi.describe())
        print('*'*20)
    

In [ ]:
##getting the bmi for every each category in Age_category variable
bmi_finder('Age_category')

In [ ]:
##getting bmi according to gender
bmi_finder('gender')

In [ ]:
stroke_df.bmi.describe()

#### <b>Conclusion drawn form analysis of 'bmi', 'Age_category' and 'Gender' variables</b>

*  Mean of the bmi variable is 28

*  Mean of categories age column that is male and female is similar

* Mean of bmi of child category is way less then other categories in Age_category variable,

  for that filling out the null values in bim column as 20 for child, 27 of youth, 30 for adult and 31 for senior

In [ ]:
##filling out the null values with mean bmi of each category
stroke_df['bmi'] = np.where((stroke_df.bmi.isnull())&(stroke_df.Age_category == 'child'),20,stroke_df.bmi)
stroke_df['bmi'] = np.where((stroke_df.bmi.isnull())&(stroke_df.Age_category == 'youth'),27,stroke_df.bmi)
stroke_df['bmi'] = np.where((stroke_df.bmi.isnull())&(stroke_df.Age_category == 'adult'),31,stroke_df.bmi)
stroke_df['bmi'] = np.where((stroke_df.bmi.isnull())&(stroke_df.Age_category == 'senior'),30,stroke_df.bmi)


In [ ]:
##univariant analysis of categorical column:
def count_plot(column_name):
    plt.figure(figsize=(8,8))
    #sns.set_theme(style ='darkgrid')
    splot = sns.countplot(data = stroke_df,x = column_name)
    for p in splot.patches:
        splot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
    plt.xlabel(column_name,fontsize = 15)
    plt.ylabel('count',fontsize=15)
    plt.title('count plot of '+column_name+'\n',fontsize = 15)

In [ ]:
def variable_classifier(df):
    cat =  [var for var in df.columns if df[var].dtypes == 'object']
    numerical = [var for var in df.columns if df[var].dtypes != 'object']
    return [cat,numerical]

In [ ]:
for var in variable_classifier(stroke_df)[0]:
    count_plot(var)

In [ ]:
#changing data types of hypertention,heart_disease,stroke
lst = ['hypertension','heart_disease','stroke']
for i in lst:
    stroke_df[i] = stroke_df[i].astype('category')

In [ ]:
##lets observe number of smokers according to gender:
stroke_df.groupby('gender')['stroke'].value_counts()

In [ ]:
plt.figure(figsize=(8,10))
plot =  sns.countplot(data = stroke_df,x = 'gender',hue = 'stroke')
for p in plot.patches:
        plot.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points') 

In [ ]:
##analysing gender,age_group,work_type,and smoking_status w.r.t stroke
col_name = ['gender','Age_category','work_type','smoking_status']
for column in col_name:
    plt.figure(figsize=(8,8))
    plot = sns.countplot(data = stroke_df,x = column,hue = 'stroke')
    for p in plot.patches:
        plot.annotate(format(p.get_height(),'.1f'),
                     (p.get_x()+p.get_width()/2.,p.get_height()),
                     ha = 'center',va = 'center',
                     xytext = (0,9),
                     textcoords = 'offset points')
        plt.xlabel(column,fontsize = 15)
        plt.ylabel('count',fontsize=15)
        plt.title('stroke and '+column+' plot'+'\n',fontsize = 15)

#### <b>Conclusions that can be drawan from bivariant analysis:</b>

->people working in private sector are more prone to stroke

-> More number of females got stroke than males

-> Seniors i.e. people above the age of 60 have got stroke more number of times than any other category

In [ ]:
##analysing numerical col 
numerical = ['age','avg_glucose_level','bmi']
for i in numerical:
    plt.figure(figsize=(8,5))
    sns.histplot(data = stroke_df,x = i,kde = True)
    plt.xlabel(i,fontsize = 15)

In [ ]:
stroke_df.groupby('Age_category').boxplot(column= 'avg_glucose_level',layout = (1,4),fontsize = 10,rot=30)
plt.show()

In [ ]:
stroke_df.groupby('Age_category').boxplot(column= 'bmi',layout = (1,4),fontsize = 10,rot=30)
plt.show()

In [ ]:
##importing libraries
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix

In [ ]:
##encoding -> label encoding for gender, Residence_type,ever_married
#dropping id
stroke_df.drop(columns='id',inplace=True)

In [ ]:
label_lst = ['gender', 'Residence_type','ever_married']
le = LabelEncoder()
for i in label_lst:
    stroke_df[i] = le.fit_transform(stroke_df[i])

In [ ]:
stroke_df.head()

Gender male = 1, female = 0

Residence_Type = Urban = 1, rural = 0

ever_married yes = 1, no = 0

In [ ]:
##one hot encoding for work_type,smoking_status,Age_category
stroke_df = pd.get_dummies(stroke_df,columns=['work_type','smoking_status','Age_category'],
                           prefix=['work_type','smoking_status','Age_cat'])

##dropping last variable of each category
stroke_df.drop(columns=['smoking_status_smokes','Age_cat_youth','work_type_Self-employed'],inplace=True)

In [ ]:
stroke_df.head()

In [ ]:
X = stroke_df.loc[:,stroke_df.columns != 'stroke']
y = stroke_df['stroke']


In [ ]:
#without validation and hyperparameter tuning 
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size= 0.3,random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
parameters = {
    'Decision_Tree':{
        'model': DecisionTreeClassifier(),
        'param':{
            'criterion':['gini','entropy'],
            'max_depth':[1,2,3,4,5,6,7,None],
            'min_samples_leaf':[1,2,3,4,5,10,20,30]
        }
    },
    'Random_Forest':{
        'model': RandomForestClassifier(),
        'param':{
            'n_estimators' : [int(x) for x in np.linspace(start = 10,stop = 100,num= 10)],
            'bootstrap': [True,False],
            'max_depth':[80,90,100,110],
            'max_features':['auto','sqrt'],
            'min_samples_leaf':[3,4,5],
            'min_samples_split':[8,10,12]
            
        }
    },
    'Logistic_Regression':{
        'model': LogisticRegression(),
        'param':{
            'C':[1,5,10]
        }
    },
    'SVM':{
        'model':SVC(gamma='auto'),
        'param':{
            'C':[1,10,20,30],
            'kernel':['rbf']
        }
    }
    
}

In [ ]:
lst = []
for key,value in parameters.items():
    clf = GridSearchCV(estimator=value['model'], param_grid=value['param'], cv = 5)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    lst.append({
        'model':key,
        'BestScore':clf.best_score_,
        'BestParameters':clf.best_params_,
        'ModelAccuracy':accuracy
    })

In [ ]:
lst

### Accuracy of Different model:

<b>Decision_Tree : 95.04%</b>

<b>Random_Forest : 94.97%</b>

<b>SVM           : 95.04%</b>

<b>Logistic_Reg  : 95.107%</b>


In [ ]:
##working with logistic reg.
model_lr = SVC(C = 1,kernel = 'rbf')
model_lr.fit(X_train,y_train)
y_pred = model_lr.predict(X_test)
confusion_matrix(y_test,y_pred)

In [ ]:
#classification report
print(classification_report(y_test,y_pred))

In [ ]:
#confusion matrix
plot_confusion_matrix(model_lr,X_test,y_test)
plt.title('confusion matrix\n')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()